# Initial Data Building: 

In [13]:
import pandas as pd

In [22]:
injuries_df = pd.read_csv('data/injuries_2024.csv')
play_by_play_df = pd.read_csv('data/play_by_play_2024.csv')

/var/folders/wx/rd0g6p9d7gn424_rsb8_3k240000gn/T/ipykernel_74324/1838363699.py:2: DtypeWarning: Columns (179,180,182,183,189,190,197,198,203,204,222,224,226,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  play_by_play_df = pd.read_csv('data/play_by_play_2024.csv')


### Injury Data Crucial Keys
- team
- gsis_id
- week

### Play by Play Data Crucial Keys
- [player_id_columns] (map to gsis_id)
- week
- home or away team

In [18]:
injuries_df.dtypes

season                        int64
game_type                    object
team                         object
week                          int64
gsis_id                      object
position                     object
full_name                    object
first_name                   object
last_name                    object
report_primary_injury        object
report_secondary_injury      object
report_status                object
practice_primary_injury      object
practice_secondary_injury    object
practice_status              object
date_modified                object
dtype: object

In [16]:
play_by_play_df[['week', 'home_team', 'away_team', 'punt_returner_player_id']].dtypes

week                        int64
home_team                  object
away_team                  object
punt_returner_player_id    object
dtype: object

In [32]:
# Find columns with '_player_id' in their names
player_id_columns = [col for col in play_by_play_df.columns if '_player_id' in col]

# Find columns with '_player_id' in their names
player_name_columns = [col for col in play_by_play_df.columns if '_player_name' in col]

# Convert necessary columns to strings to avoid data type mismatch
injuries_df['gsis_id'] = injuries_df['gsis_id'].astype(str)
injuries_df['team'] = injuries_df['team'].astype(str)

play_by_play_df['home_team'] = play_by_play_df['home_team'].astype(str)
play_by_play_df['away_team'] = play_by_play_df['away_team'].astype(str)

# Initialize an empty list to store the merge results
merged_data_list = []

for player_col in player_id_columns:

    play_by_play_df[player_col] = play_by_play_df[player_col].astype(str)

# Iterate over the player_id_columns and merge with each one
for player_col in player_id_columns:
    merged_data = injuries_df.merge(
        play_by_play_df,
        how='inner',
        left_on=['gsis_id', 'week', 'team'],
        right_on=[player_col, 'week', 'home_team']
    )
    merged_data_list.append(merged_data)

    merged_data_away = injuries_df.merge(
        play_by_play_df,
        how='inner',
        left_on=['gsis_id', 'week', 'team'],
        right_on=[player_col, 'week', 'away_team']
    )
    merged_data_list.append(merged_data_away)

# Combine all the results into a single DataFrame
final_merged_data = pd.concat(merged_data_list).drop_duplicates().reset_index(drop=True)

In [34]:
final_merged_data[['team', 'week', 'gsis_id', 'full_name', 'report_primary_injury', *player_id_columns, *player_name_columns]]

pd.set_option('display.max_columns', None)

In [35]:
final_merged_data[['team', 'week', 'gsis_id', 'full_name', 'report_primary_injury', *player_id_columns, *player_name_columns]]

,team,week,gsis_id,full_name,report_primary_injury,td_player_id,passer_player_id,receiver_player_id,rusher_player_id,lateral_receiver_player_id,lateral_rusher_player_id,lateral_sack_player_id,interception_player_id,lateral_interception_player_id,punt_returner_player_id,lateral_punt_returner_player_id,kickoff_returner_player_id,lateral_kickoff_returner_player_id,punter_player_id,kicker_player_id,own_kickoff_recovery_player_id,blocked_player_id,tackle_for_loss_1_player_id,tackle_for_loss_2_player_id,qb_hit_1_player_id,qb_hit_2_player_id,forced_fumble_player_1_player_id,forced_fumble_player_2_player_id,solo_tackle_1_player_id,solo_tackle_2_player_id,assist_tackle_1_player_id,assist_tackle_2_player_id,assist_tackle_3_player_id,assist_tackle_4_player_id,tackle_with_assist_1_player_id,tackle_with_assist_2_player_id,pass_defense_1_player_id,pass_defense_2_player_id,fumbled_1_player_id,fumbled_2_player_id,fumble_recovery_1_player_id,fumble_recovery_2_player_id,sack_player_id,half_sack_1_player_id,half_sack_2_player_id,penalty_player_id,safety_player_id,fantasy_player_id,td_player_name,passer_player_name,receiver_player_name,rusher_player_name,lateral_receiver_player_name,lateral_rusher_player_name,lateral_sack_player_name,interception_player_name,lateral_interception_player_name,punt_returner_player_name,lateral_punt_returner_player_name,kickoff_returner_player_name,lateral_kickoff_returner_player_name,punter_player_name,kicker_player_name,own_kickoff_recovery_player_name,blocked_player_name,tackle_for_loss_1_player_name,tackle_for_loss_2_player_name,qb_hit_1_player_name,qb_hit_2_player_name,forced_fumble_player_1_player_name,forced_fumble_player_2_player_name,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_name,assist_tackle_2_player_name,assist_tackle_3_player_name,assist_tackle_4_player_name,tackle_with_assist_1_player_name,tackle_with_assist_2_player_name,pass_defense_1_player_name,pass_defense_2_player_name,fumbled_1_player_name,fumbled_2_player_name,fumble_recovery_1_player_name,fumble_recovery_2_player_name,sack_player_name,half_sack_1_player_name,half_sack_2_player_name,penalty_player_name,safety_player_name,fantasy_player_name
0,ATL,1,00-0036970,Kyle Pitts,NaN,00-0036970,00-0029604,00-0036970,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,00-0036970,K.Pitts,K.Cousins,K.Pitts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K.Pitts
1,BUF,1,00-0034857,Josh Allen,Left Hand,00-0034857,nan,nan,00-0034857,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,00-0034857,J.Allen,NaN,NaN,J.Allen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J.Allen
2,BUF,1,00-0034857,Josh Allen,Left Hand,00-0034857,nan,nan,00-0034857,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,00-0034857,J.Allen,NaN,NaN,J.Allen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J.Allen
3,CLE,1,00-0036407,Jerry Jeudy,NaN,00-0036407,00-0033537,00-0036407,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,00-0036407,J.Jeudy,D.Watson,J.Jeudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J.Jeudy
4,NO,1,00-0034981,Foster Moreau,Head,00-0034981,00-0031280,00-0034981,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan

In [28]:
# Search for rows with gsis_id '00-0039521'
gsis_id_search = '00-0037746'

# Filter the final_merged_data DataFrame
result = final_merged_data[final_merged_data['gsis_id'] == gsis_id_search]

In [29]:
result

,season_x,game_type,team,week,gsis_id,position,full_name,first_name,last_name,report_primary_injury,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
29,2024,REG,WAS,5,00-0037746,RB,Brian Robinson,Brian,Robinson,Knee,...,0,0,0.955465,NaN,NaN,NaN,NaN,NaN,0.382364,-38.236424
30,2024,REG,WAS,5,00-0037746,RB,Brian Robinson,Brian,Robinson,Knee,...,0,0,0.934548,NaN,NaN,NaN,NaN,NaN,0.249561,-24.956065
955,2024,REG,WAS,2,00-0037746,RB,Brian Robinson,Brian,Robinson,NaN,...,1,1,-0.314723,1.492175,11.468515,10.0,0.363476,0.150233,0.725306,27.469409
956,2024,REG,WAS,2,00-0037746,RB,Brian Robinson,Brian,Robinson,NaN,...,0,1,-0.400476,1.135354,8.826999,9.0,0.394103,0.056790,0.667913,33.208650
957,2024,REG,WAS,2,00-0037746,RB,Brian Robinson,Brian,Robinson,NaN,...,0,1,-0.429590,0.393320,2.773821,2.0,0.128823,0.071507,0.960237,3.976303
1889,2024,REG,WAS,2,00-0037746,RB,Brian Robinson,Brian,Robinson,NaN,...,0,1,0.257019,NaN,NaN,NaN,NaN,NaN,0.460636,-46.063563
1890,2024,REG,WAS,2,00-0037746,RB,Brian Robinson,Brian,Robinson,NaN,...,0,1,-0.194900,NaN,NaN,NaN,NaN,NaN,0.361427,-36.142695
1891,2024,REG,WAS,2,00-0037746,RB,Brian Robinson,Brian,Robinson,NaN,...,0,1,-0.347084,NaN,NaN,NaN,NaN,NaN,0.463976,-46.397579
1892,2024,REG,WAS,2,00-0037746,RB,Brian Robinson,Brian,Robinson,NaN,...,0,1,2.068856,NaN,NaN,NaN,NaN,NaN,0.213691,-21.369092
1893,2024,REG,WAS,2,00-0037746,RB,Brian Robinson,Brian,Robinson,NaN,...,0,1,-0.437093,NaN,NaN,NaN,NaN,NaN,0.540601,-54.060060


In [3]:
print(f"Columns in injuries_2024.csv ({len(injuries_df.columns)} columns):")
print(injuries_df.columns.tolist())
print(f"\nColumns in play_by_play_2024.csv ({len(play_by_play_df.columns)} columns):")
print(play_by_play_df.columns.tolist())

Columns in injuries_2024.csv (16 columns):
['season', 'game_type', 'team', 'week', 'gsis_id', 'position', 'full_name', 'first_name', 'last_name', 'report_primary_injury', 'report_secondary_injury', 'report_status', 'practice_primary_injury', 'practice_secondary_injury', 'practice_status', 'date_modified']

Columns in play_by_play_2024.csv (372 columns):
['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team', 'season_type', 'week', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date', 'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_con

In [4]:
print(f"Number of rows in play_by_play: {play_by_play_df.shape[0]}")
print(f"Number of rows in injuries: {injuries_df.shape[0]}")

Number of rows in play_by_play: 15913
Number of rows in injuries: 1617


In [5]:
common_columns = set(injuries_df.columns).intersection(play_by_play_df.columns)

if common_columns:
    print(f"Common columns between injuries_2024.csv and play_by_play_2024.csv ({len(common_columns)} columns):")
    print(common_columns)
else:
    print("There are no common columns between injuries_2024.csv and play_by_play_2024.csv.")

Common columns between injuries_2024.csv and play_by_play_2024.csv (2 columns):
{'week', 'season'}


In [6]:
player_id_columns = [col for col in play_by_play_df.columns if '_player_id' in col]

if player_id_columns:
    print(f"Columns containing '_player_id' in play_by_play_2024.csv ({len(player_id_columns)} columns):")
    print(player_id_columns)
else:
    print("No columns containing '_player_id' in play_by_play_2024.csv.")

Columns containing '_player_id' in play_by_play_2024.csv (43 columns):
['td_player_id', 'passer_player_id', 'receiver_player_id', 'rusher_player_id', 'lateral_receiver_player_id', 'lateral_rusher_player_id', 'lateral_sack_player_id', 'interception_player_id', 'lateral_interception_player_id', 'punt_returner_player_id', 'lateral_punt_returner_player_id', 'kickoff_returner_player_id', 'lateral_kickoff_returner_player_id', 'punter_player_id', 'kicker_player_id', 'own_kickoff_recovery_player_id', 'blocked_player_id', 'tackle_for_loss_1_player_id', 'tackle_for_loss_2_player_id', 'qb_hit_1_player_id', 'qb_hit_2_player_id', 'forced_fumble_player_1_player_id', 'forced_fumble_player_2_player_id', 'solo_tackle_1_player_id', 'solo_tackle_2_player_id', 'assist_tackle_1_player_id', 'assist_tackle_2_player_id', 'assist_tackle_3_player_id', 'assist_tackle_4_player_id', 'tackle_with_assist_1_player_id', 'tackle_with_assist_2_player_id', 'pass_defense_1_player_id', 'pass_defense_2_player_id', 'fumbled_

In [7]:
total_matches = 0 

for col in player_id_columns:
    
    matching_rows = play_by_play_df[play_by_play_df[col].isin(injuries_df['gsis_id'])]
    
    
    num_matches = matching_rows.shape[0]
    
    if num_matches > 0:
        print(f"Matches found in column '{col}': {num_matches} rows")
        print(matching_rows[[col]])
        total_matches += num_matches  
    else:
        print(f"No matches found in column '{col}'.")


print(f"Total number of matched rows across all columns: {total_matches}")

Matches found in column 'td_player_id': 255 rows
      td_player_id
72      00-0034857
87      00-0033555
104     00-0037261
128     00-0034857
184     00-0039894
...            ...
15521   00-0032211
15528   00-0029892
15558   00-0036411
15594   00-0037545
15626   00-0033906

[255 rows x 1 columns]
Matches found in column 'passer_player_id': 1779 rows
      passer_player_id
3           00-0035228
4           00-0035228
8           00-0035228
9           00-0035228
13          00-0035228
...                ...
15334       00-0038579
15345       00-0038579
15346       00-0038579
15347       00-0038579
15348       00-0038579

[1779 rows x 1 columns]
Matches found in column 'receiver_player_id': 3379 rows
      receiver_player_id
8             00-0035500
9             00-0039849
19            00-0033555
24            00-0037744
27            00-0035500
...                  ...
15866         00-0030061
15871         00-0030061
15875         00-0033591
15882         00-0036550
15884        

1. check if data that matches with gsis_id 

In [3]:
import pandas as pd
play_df = pd.read_csv("data/play_by_play_2024.csv")
injury_df = pd.read_csv("data/injuries_2024.csv")
pd.set_option('display.max_columns', None)

/var/folders/rx/0shr4jt1365bgrnlk6fkmjjh0000gn/T/ipykernel_42168/3373673293.py:2: DtypeWarning: Columns (179,180,182,183,189,190,197,198,203,204,222,224,226,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  play_df = pd.read_csv("data/play_by_play_2024.csv")


In [4]:
injury_df.head()

,season,game_type,team,week,gsis_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified
0,2024,REG,ARI,1,00-0035629,T,Jonah Williams,Jonah,Williams,Knee,NaN,Out,NaN,NaN,\n,2024-09-08T18:35:57Z
1,2024,REG,ARI,1,00-0039521,WR,Xavier Weaver,Xavier,Weaver,Oblique,NaN,Out,Oblique,NaN,Did Not Participate In Practice,2024-09-06T19:05:30Z
2,2024,REG,ARI,1,00-0037141,S,Joey Blount,Joey,Blount,Back,NaN,Questionable,Back,NaN,Limited Participation in Practice,2024-09-06T19:05:21Z
3,2024,REG,ARI,1,00-0039737,TE,Tip Reiman,Tip,Reiman,Ankle,NaN,Questionable,Ankle,NaN,Limited Participation in Practice,2024-09-06T19:05:21Z
4,2024,REG,ATL,1,00-0031583,DT,Grady Jarrett,Grady,Jarrett,NaN,NaN,NaN,Not injury related - resting player,NaN,Did Not Participate In Practice,2024-09-06T18:35:04Z


In [5]:
injury_df["date"] = pd.to_datetime(injury_df['date_modified'])
play_df["date"] = pd.to_datetime(play_df['game_date'])
injury_df['date'] = injury_df['date'].dt.tz_localize(None)
play_df['date'] = play_df['date'].dt.tz_localize(None)


In [6]:
plays_with_injuries = play_df[play_df['desc'].str.contains("was injured", na=False)]

In [8]:
pattern = r'(\w+\.(\w|-|\.|\')+)(?: \w+)? was injured'
# Extract the injured player's name from the desc column
injured_players = plays_with_injuries.loc[:, "desc"].str.extract(pattern)

In [9]:
plays_with_injuries = pd.concat([plays_with_injuries, injured_players], axis=1)
plays_with_injuries.rename(columns={0: "injured_player"}, inplace=True)

In [10]:
plays_with_injuries.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

# Select Data: 

# Clean Data: 

# Construct Data: 

# Integrate Data: 

# Format Data: 